In [83]:
import numpy as np
import pandas as pd

Загрузите датасет `digits` с помощью функции `load_digits` из `sklearn.datasets` и подготовьте матрицу признаков X и ответы на обучающей выборке y (вам потребуются поля `data` и `target` в объекте, который возвращает `load_digits`).

In [84]:
from sklearn.datasets import load_digits
digits = load_digits()

Для оценки качества далее нужно будет использовать `cross_val_score` из `sklearn.cross_validation` с параметром `cv=10`. Эта функция реализует `k-fold cross validation` c `k` равным значению параметра `cv`. Мы предлагаем использовать `k=10`, чтобы полученные оценки качества имели небольшой разброс, и было проще проверить полученные ответы. На практике же часто хватает и `k=5`. Функция `cross_val_score` будет возвращать `numpy.ndarray`, в котором будет `k` чисел - качество в каждом из `k` экспериментов `k-fold cross validation`. Для получения среднего значения (которое и будет оценкой качества работы) вызовите метод `.mean()` у массива, который возвращает `cross_val_score`.

In [85]:
from sklearn.cross_validation import cross_val_score
cv = 10

def cross_val_score_cv(*args, **kwargs):
    return cross_val_score(*args, cv=cv, **kwargs)

С небольшой вероятностью вы можете натолкнуться на случай, когда полученное вами качество в каком-то из пунктов не попадет в диапазон, заданный для правильных ответов - в этом случае попробуйте перезапустить ячейку с `cross_val_score` несколько раз и выбрать наиболее «типичное» значение. Если это не помогает, то где-то была допущена ошибка.

Если вам захочется ускорить вычисление `cross_val_score` - можете попробовать использовать параметр `n_jobs`, но будьте осторожны: в одной из старых версий `sklearn` была ошибка, которая приводила к неверному результату работы `cross_val_score` при задании `n_jobs` отличным от 1. Сейчас такой проблемы возникнуть не должно, но проверить, что все в порядке, не будет лишним.

In [86]:
def write_answ(n, res):
    with open('beg_for_{}.txt'.format(n), 'w') as out:
        out.write(str(res))


### task 1
Создайте `DecisionTreeClassifier` с настройками по умолчанию и измерьте качество его работы с помощью `cross_val_score`. Эта величина и будет ответом в пункте 1.

In [87]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
estimator = dtc.fit(digist.data, digist.target)

res = cross_val_score_cv(estimator, digist.data, y=digist.target).mean()
print res

write_answ(1, res)

0.825881325812


### task 2

Воспользуйтесь `BaggingClassifier` из `sklearn.ensemble`, чтобы обучить бэггинг над `DecisionTreeClassifier`. Используйте в `BaggingClassifier` параметры по умолчанию, задав только количество деревьев равным 100.

Качество классификации новой модели - ответ в пункте 2. Обратите внимание, как соотносится качество работы композиции решающих деревьев с качеством работы одного решающего дерева.



In [88]:
from sklearn.ensemble import BaggingClassifier
bg = BaggingClassifier(estimator, n_estimators=100)

res = cross_val_score_cv(bg, digist.data, y=digist.target).mean()
print res

write_answ(2, res)

0.921565669902


### task 3

Теперь изучите параметры `BaggingClassifier` и выберите их такими, чтобы каждый базовый алгоритм обучался не на всех $d$ признаках, а на $\sqrt d$ случайных признаков. Качество работы получившегося классификатора - ответ в пункте 3. Корень из числа признаков - часто используемая эвристика в задачах классификации, в задачах регрессии же часто берут число признаков, деленное на три. Но в общем случае ничто не мешает вам выбирать любое другое число случайных признаков.

In [89]:
# print np.sqrt(digist.data.shape[0])
bg = BaggingClassifier(estimator, n_estimators=100, max_features=int(np.sqrt(digist.data.shape[1])))

res = cross_val_score_cv(bg, digist.data, y=digist.target).mean()
print res

write_answ(3, res)

0.926628324932


### task 4

Наконец, давайте попробуем выбирать случайные признаки не один раз на все дерево, а при построении каждой вершины дерева. Сделать это несложно: нужно убрать выбор случайного подмножества признаков в `BaggingClassifier` и добавить его в `DecisionTreeClassifier`. Какой параметр за это отвечает, можно понять из документации `sklearn`, либо просто попробовать угадать (скорее всего, у вас сразу получится). Попробуйте выбирать опять же $\sqrt d$ признаков. Качество полученного классификатора на контрольной выборке и будет ответом в пункте 4.

In [96]:
dtc = DecisionTreeClassifier(max_features=int(np.sqrt(digist.data.shape[1])))
bg = BaggingClassifier(dtc, n_estimators=100)

res = cross_val_score_cv(bg, digist.data, y=digist.target).mean()
print res

write_answ(4, res)

0.947217405526


### task 5

Полученный в пункте 4 классификатор - бэггинг на рандомизированных деревьях (в которых при построении каждой вершины выбирается случайное подмножество признаков и разбиение ищется только по ним). Это в точности соответствует алгоритму `Random Forest`, поэтому почему бы не сравнить качество работы классификатора с `RandomForestClassifier` из `sklearn.ensemble`. Сделайте это, а затем изучите, как качество классификации на данном датасете зависит от количества деревьев, количества признаков, выбираемых при построении каждой вершины дерева, а также ограничений на глубину дерева. Для наглядности лучше построить графики зависимости качества от значений параметров, но для сдачи задания это делать не обязательно.

На основе наблюдений выпишите через пробел номера правильных утверждений из приведенных ниже в порядке возрастания номера (это будет ответ в п.5)

1. Случайный лес сильно переобучается с ростом количества деревьев
1. При очень маленьком числе деревьев (5, 10, 15), случайный лес работает хуже, чем при большем числе деревьев
1. С ростом количества деревьев в случайном лесе, в какой-то момент деревьев становится достаточно для высокого качества классификации, а затем качество существенно не меняется.
1. При большом количестве признаков (для данного датасета - 40, 50) качество классификации становится хуже, чем при малом количестве признаков (5, 10). Это связано с тем, что чем меньше признаков выбирается в каждом узле, тем более различными получаются деревья (ведь деревья сильно неустойчивы к изменениям в обучающей выборке), и тем лучше работает их композиция.
1. При большом количестве признаков (40, 50, 60) качество классификации лучше, чем при малом количестве признаков (5, 10). Это связано с тем, что чем больше признаков - тем больше информации об объектах, а значит алгоритм может делать прогнозы более точно.
1. При небольшой максимальной глубине деревьев (5-6) качество работы случайного леса намного лучше, чем без ограничения глубины, т.к. деревья получаются не переобученными. С ростом глубины деревьев качество ухудшается.
1. При небольшой максимальной глубине деревьев (5-6) качество работы случайного леса заметно хуже, чем без ограничений, т.к. деревья получаются недообученными. С ростом глубины качество сначала улучшается, а затем не меняется существенно, т.к. из-за усреднения прогнозов и различий деревьев их переобученность в бэггинге не сказывается на итоговом качестве (все деревья преобучены по-разному, и при усреднении они компенсируют переобученность друг-друга).

In [103]:
result = {
    1: None,
    2: None,
    3: None,
    4: None,
    5: None,
    6: None,
    7: None,
}

In [107]:
from sklearn.ensemble import RandomForestClassifier

for tree_n in (5, 10, 15, 25, 50, 100, 500):
    rf = RandomForestClassifier(n_estimators=tree_n)
    print tree_n, cross_val_score_cv(rf, digist.data, y=digist.target).mean()

5 0.888222199956
10 0.904360824518
15 0.939996112251
25 0.943251447413
50 0.948886874304
100 0.953341050637
500 0.955582820161


In [124]:
result[1] = False
result[2] = True
result[3] = True

In [111]:
for features_n in (5, 10, 15, 40, 50, 60):
    rf = RandomForestClassifier(max_features=features_n)
    print features_n, cross_val_score_cv(rf, digist.data, y=digist.target).mean()

5 0.901500479471
10 0.923805183183
15 0.934344203708
40 0.914974992297
50 0.916622552883
60 0.899845957091


In [123]:
result[4] = True
result[5] = False

In [121]:
for depth in (5, 6, 15, 40, 50, 60):
    rf = RandomForestClassifier(max_depth=depth)
    print depth, cross_val_score_cv(rf, digist.data, y=digist.target).mean()

rf = RandomForestClassifier()
print cross_val_score_cv(rf, digist.data, y=digist.target).mean()

5 0.874093547532
6 0.884934389255
15 0.912670551584
40 0.92003029561
50 0.915437828043
60 0.914217885176
0.923182498213


In [122]:
result[6] = False
result[7] = True

In [125]:
write_answ(5, ' '.join([str(key) for key, value in result.iteritems() if value]))